# Problem Description.

The task of this project is to solve the
Japanese grid puzzle [Yosenabe](http://www.nikoli.co.jp/en/puzzles/yosenabe.html)
using ASP.
To illustrate the rules of the game, consider a grid as follows:
<img width="230" height="150" src="img/yosenabe.png">

Given such a grid, initially without arrows,
the task is to move each number surrounded by a frame into one
of the gray areas along a straight line, respecting the following conditions:
* The straight lines of any two moved numbers 
  must not cross or meet at any grid cell.
* Each gray area must be populated with at least one moved number.
* An area may be associated with a positive goal number, shown within it.
  If this is the case, then the numbers moved into the area must sum up exactly to the goal number.

The unique solution of the example, indicated by the arrows of the image,
fulfills these conditions:
* The straight lines do not cross or meet.
* Some number is moved into each of the four gray areas.
* The bottom left area is associated with goal number ``6``, 
  and the numbers ``5`` and ``1``, that add up to ``6``, are moved to it.
  The top right area is associated with goal number ``4``, 
  and exactly number ``4`` is moved to it.

Note that the displayed solution is unique.
One could try to move the two numbers ``2`` into the top right area, but 
that would leave the bottom right area unoccupied.

A number can be moved across an area.
For instance, in our example the number ``4`` crosses the area in the middle.
On the other hand, you may always assume that the move of a number stops at the first cell (with respect to its direction)
of the area to which the number is moved.
For instance, number ``1`` is moved to the cell ``(2,1)``, and 
there would be no solution if we moved it further into the area until the cell ``(1,1)``.
<!--
That is, regardless of the cell to which number ``5`` is moved,
it cannot help to move ``1`` to the cell in the lower left corner
as the move enters the same area already before.
-->
You can play the game [here](http://www.janko.at/Raetsel/Yosenabe/).
It is fun and it will help you to get used to the rules.

# Representation in ASP.

A Yosenabe puzzle is represented by facts of the following predicates:
```
cell(X,Y).         % the cell (X,Y) belongs to the grid
area(X,Y,A).       % the cell (X,Y) belongs to area A
number(X,Y,N).     % the cell (X,Y) initially contains number N
goal(A,G).         % the goal number of area A is G
```

The example shown before is represented by the following facts:
```
cell(1,1).    cell(1,2). ...  cell(5,4).  cell(5,5).  number(1,5,5).
area(1,1,1).  area(2,1,1).                goal(1,6).  number(3,1,1).
area(2,3,2).  area(3,3,2).  area(4,3,2).              number(3,4,2).
area(3,5,3).  area(4,5,3).  area(5,5,3).  goal(3,4).  number(4,2,4).
area(5,1,4).                                          number(5,3,2).
```

A solution is represented by atoms of the predicate ``target/4``:
```
target(X,Y,XX,YY).  % the number in the cell (X,Y) is moved to the cell (XX,YY)
```

The solution of the example consists of the following atoms:
```
target(1,5,1,1) target(3,1,2,1) target(3,4,3,3) target(4,2,4,5) target(5,3,5,1)
```

# Task.

Write an ASP encoding such that given an instance,
the stable models of the encoding and the instance
correspond to the solutions of the problem represented by the instance.

# Framework.

In the directory ``instances`` you can find 14 instances (our example is ``instances/instance01.lp``). You have to submit a file named ``yosenabe.lp``, included as a template in this directory, that contains the following line (and no more ``#show`` statements) so that in the output only the atoms of predicate ``target/4`` appear:

``#show target/4.``

Click the link at the following cell to download a zip file with the instances and the template. We recommend you to work with them in your own computer, using your own installation of ``clingo``.


In [1]:
from IPython.display import FileLink
FileLink("yosenabe.zip")

/home/jovyan/projects/yosenabe/yosenabe.zip

Alternatively, you can also run your encoding in the next cell, but this is not the recommended option. If you work in this notebook, remember to download the files that you modify to your computer, otherwise you will lose your changes.

In [57]:
%%clingo 3 instances/instance01.lp -

%# Legend
%# cell(X,Y).         % the cell (X,Y) belongs to the grid
%# area(X,Y,A).       % the cell (X,Y) belongs to area A
%# number(X,Y,N).     % the cell (X,Y) initially contains number N
%# goal(A,G).         % the goal number of area A is G

dir(0,1). dir(0,-1). dir(1,0). dir(-1,0).

%# Hints:
%# - You may use the gaps of adjacent cell coordinates to denote directions.
%# - You can use the "--text" option of clingo to inspect the ground program.

%# pick a direction for each number at the beginning.
%# once a direction is chosen, the number has to move in that direction for the duration of the solution.
%# track original coordinates with N - we need them for target and also as ID, in case there are duplicate numbers
1 { number_dir(X,Y,N,LR,UD) : dir(LR,UD) } 1 :- number(X,Y,N).

%# make a move(X,Y,XX,YY,N) for each number(X,Y,N)
move(X,Y,X,Y,N) :- number(X,Y,N).

%# maybe? move each number to a new location
0 { move(X, Y, XX + LR, YY + UD, N) } 1 :-  %# choose to move or not
    number_dir(X, Y, N, LR, UD),            %# for a given number direction
    move(X,Y,XX,YY,N),                      %# if there's a move to (XX,YY)
    %# not move(X,Y,XX + LR, YY + UD, N),   %# AND there isn't a move to the potential destination already
    cell(XX + LR,YY + UD).                  %# AND the destination exists

%# Find the current sum_area(A,S)
sum_area(A,S) :-
    area(_,_,A),
    S = #sum { N,AA :
        A == AA,
        area(XX,YY,AA),
        move(X,Y,XX,YY,N),
        number_dir(X, Y, N, LR, UD),
        not move(X, Y, XX + LR, YY + UD, N) }.

n_goals(NG) :- NG = #count { A,G : goal(A,G) }. %# number of goals
n_areas(NA) :- NA = #count { A : area(_,_,A) }.

%# TODO: I think I could formulate latest_move as a rule

area_populated(A) :-
    area(XX,YY,A),
    move(X,Y,XX,YY,N),
    number_dir(X, Y, N, LR, UD),
    not move(X, Y, XX + LR, YY + UD, N).

areas_populated("true") :-
    n_areas(NA),
    NA #count { A : area_populated(A) }.

%# goal_achieved(A) :-
%#     goal(A,G),
%#    SumArea

goals_achieved("true") :-
    n_goals(NG),
    NG #count { G : goal(A,G), sum_area(A,S), S == G } NG. %# count of solved goals

%# Encode winning!
%# Every area has one or more numbers
%# Every goal is achieved.
solved("true") :- areas_populated("true"), goals_achieved("true").

solved("false") :- not solved("true").

%# if solved, then set targets
target(X, Y, XX, YY) :-
    solved("true"),
    number_dir(X, Y, N, LR, UD),
    move(X, Y, XX, YY, N),
    not move(X, Y, XX + LR, YY + UD, N). %# and we don't need to check if the destination exists

%# This should prune solutions where different numbers share cells
:- #count { XX,YY : move(_,_,XX,YY,_) } 1.

%# This prunes non-solutions
:- solved("false").

%# --- Terminating conditions---
%# Note: it's not necessary to have the number land in an area.
%# (1) The straight lines of any two moved numbers must not cross or meet at any grid cell.
%# --- OK, this is the simplest one. It cannot be the case that two numbers went through the same cell at any time.
%# (2) Each gray area must be populated with at least one moved number.
%# (3) An area may be associated with a positive goal number, shown within it.

#show area_populated/1.
%#show ok/1.
%#show solved/1.
#show target/4.
%#show number/3.
%#show number_dir/5.
%#show area/3.
%#show move/5.
% Answer: target(3,1,2,1) target(3,4,3,3) target(5,3,5,1) target(4,2,4,5) target(1,5,1,1)

clingo version 5.8.0
Reading from instances/instance01.lp ...
Solving...
Answer: 1 (Time: 0.013s)
area_populated(3) area_populated(2) area_populated(1) area_populated(4) target(1,5,1,1) target(3,1,2,1) target(3,4,3,3) target(4,2,4,5) target(5,3,5,1)
Answer: 2 (Time: 0.013s)
area_populated(3) area_populated(2) area_populated(1) area_populated(4) target(1,5,1,1) target(3,1,1,1) target(3,4,3,3) target(4,2,4,5) target(5,3,5,1)
SATISFIABLE

Models       : 2
Calls        : 1
Time         : 0.013s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.011s


# Formalities.
You can work on the solution alone or in groups of two people. 
Different groups have to submit different solutions, in case
of plagiarism all groups involved will fail the project. 

We will test your encoding with the 14 provided instances.
Your solution has to correctly encode all solutions for every instance
(in this case, all instances have a unique solution).
This will be tested automatically. 
Please contact us if you get stuck.

# Tips:
      
* The directions between adjacent cells (up, down, left, and right) are 
entirely symmetric and vary only by coordinate differences between the cells. 
Please avoid code duplication due to referring to directions more specifically than needed. 
It is not a good idea to distinguish directions by using atoms such as 
``goUp(X,Y,X,Y+1)``, ``goDown(X,Y,X,Y-1)``,
``goRight(X,Y,X+1,Y)``, and ``goLeft(X,Y,X-1,Y)``.
On the other hand, it is a good idea to use atoms like 
``go(X,Y,X,Y+1,0,1)``, ``go(X,Y,X,Y-1,0,-1)``, ``go(X,Y,X+1,Y,1,0)``, and
``go(X,Y,X-1,Y,-1,0)`` because they can be treated uniformly within rules.      

* If you are stuck you can contact us. We will do out best to answer all your questions. You can send us questions and remarks either via Moodle or by email.

* Start as soon as possible to avoid running out of time. However, if you still realize that you have problems making it before the deadline, please contact us instead of copying another solution.
